## Lab 3. Peak Load Manager

### Introduction

In this notebook we show you how to create another agent on Amazon Bedrock.

This agent will be a peak load manager agent, identifying non-essential processes that can be shifted to off-peak hours.

The following represents the piece of architecture that will be built on this module.

![Architecture](../img/lab3-architecture.png)

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/botocore-1.35.55-py3-none-any.whl \
    ../boto3/boto3-1.35.55-py3-none-any.whl \
    ../boto3/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

Restart kernel for packages to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [2]:
import boto3
import os
import sys

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

agent_foundation_model = ['anthropic.claude-3-5-sonnet-20240620-v1:0']
#agent_foundation_model = ['anthropic.claude-3-5-haiku-20241022-v1:0']

In [3]:
peak_agent_name = f"peak-agent-{account_id}"

peak_lambda_name = f"fn-peak-agent-{account_id}"

peak_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{peak_agent_name}'

On following section, we're adding `agents.py` on Python path, so the file can be recognized and invoked.

In [4]:
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

parent_dir = os.path.dirname(current_dir)
#print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

Now, you're going to import from helper file `agents.py` AgentsForAmazonBedrock helper class.

This class is a helper totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by this class.

Following are methods that you're going to invoke on this lab:

- create_agent: Create a new agent and respective IAM roles
- add_action_group_with_lambda: Create a lambda function and add it as an action group for a previous created agent
- create_agent_alias: Create an alias for this agent
- invoke: Execute agent

In [5]:
from agent import AgentsForAmazonBedrock

agents = AgentsForAmazonBedrock()

Create the peak load manager agent

In [48]:
peak_agent_id = agents.create_agent(
    peak_agent_name, 
    """
        You are a peak load manager bot. 
        You can retrieve information from IoT devices, identify process and their consumption and suggest shifts to off-peak hours.
    """, 
    """
        You are a peak load manager bot. 
        You can retrieve information from IoT devices, identify process and their consumption and suggest shifts to off-peak hours.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 

    """,
    agent_foundation_model
)

peak_agent_id

'H51N1PCTZ1'

### Creating Lambda

On this block, we're going to generate Lambda function Code and associate it with created agent.

In [49]:
%%writefile peak_load.py
import json
import random

PEAK_LIST = ['fridge','bathtub','iron','dry-machine','car charger socket','light bulbs','gardening electrical system']

def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def get_consumption_info(customer_id):
    # TODO: Implement real business logic

    return [
        {
         "monthly_avg": "150kw",
         "monthly_p90": "140kw",
         "monthly_max": "440kw",
         "monthly_mix": "90kw",
         "peak_load_day": "55kw",
         "price_per_kw": '%.2f'%(random.uniform(1.0, 20.0)),
         "peak_item": PEAK_LIST[random.randint(0, 6)]
        }
    ]

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    
    if function == 'get_consumption_info':
        customer_id = get_named_parameter(event, "customer_id")
        result = get_consumption_info(customer_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Writing peak_load.py


Add lambda as an action group for this agent and prepare it.

In [50]:
resp = agents.add_action_group_with_lambda(
            peak_agent_name, 
            peak_lambda_name, 
            "peak_load.py", 
            [
                {
                    "name": "get_consumption_info",
                    "description": """Retrieves usage information like price,  
                                      avg, p90, peak_load""",
                    "parameters": {
                                    "customer_id": {
                                        "description": "The ID of the customer",
                                        "required": True,
                                        "type": "string"
                                    }
                                }
                }
            ], 
            "forecast_consumption_actions", 
            "Function to get usage forecast for a user "
            )

### Create alias

Create an alias that will be used further on multi-agent collaborator feature:

In [51]:
agents_boto3 = boto3.client('bedrock-agent',
                            region_name=region)

peak_agent_alias_resp = agents_boto3.create_agent_alias(agentId=peak_agent_id, 
                                                              agentAliasName='v1')

In [ ]:
# create alias
#consumption_agent_alias_arn = agents.create_agent_alias(consumption_agent_id, 'v1')

### Invoking Agent

Now, let's run some tests on agent to make sure it's working:

In [52]:
%%time
print(agents.invoke("What's causing my peak load and how much I've spent on it? My id is 45", 
                    peak_agent_id)
)

Based on the information retrieved, your peak load is primarily caused by your gardening electrical system. Your monthly maximum consumption reached 440kW, with a daily peak load of 55kW. The price per kW is $3.71, so your peak consumption is costing you significantly more than your average usage. To calculate the exact amount spent on peak load, we'd need more detailed billing information, but it's clear that reducing the usage of your gardening electrical system during peak hours could lead to substantial savings.
CPU times: user 29.4 ms, sys: 4.25 ms, total: 33.7 ms
Wall time: 10.1 s


Store environment variables to be used on next notebooks.

In [ ]:
peak_agent_arn = agents.get_agent_arn_by_name(peak_agent_name)
peak_agent_alias_arn = peak_agent_alias_resp['agentAlias']['agentAliasArn']
peak_agent_alias_id = peak_agent_alias_resp['agentAlias']['agentAliasId']

%store peak_agent_arn
%store peak_agent_alias_arn
%store peak_agent_alias_id
%store peak_lambda_name
%store peak_agent_name
%store peak_agent_id

In [54]:
peak_agent_arn, peak_agent_alias_arn, peak_agent_alias_id

('arn:aws:bedrock:us-west-2:471112746845:agent/H51N1PCTZ1',
 'arn:aws:bedrock:us-west-2:471112746845:agent-alias/H51N1PCTZ1/ZMESLWDRP5',
 'ZMESLWDRP5')

### Clean Up

In [5]:
agents.delete_lambda(peak_lambda_name)

In [ ]:
agents.delete_agent_alias(peak_agent_alias_id, peak_agent_id)

In [ ]:
agents.delete_agent(peak_agent_name)